# Análise de Fatores de Risco para AVC
## 1. Introdução
- Breve descrição do problema e objetivos do projeto.
- Fonte dos dados (link e descrição).
- Tecnologias utilizadas.

## 2. Importação das Bibliotecas e do Dataset

In [2]:
import pandas as pd
import matplotlib as plt

## 3. Entendimento Inicial dos Dados
- shape, head(), info(), describe()
- Verificação de tipos de dados, valores únicos, nulos, etc.

In [3]:
#importando dados do arquivo raw
url = 'https://raw.githubusercontent.com/danoliveiraed/portfolio-projects/refs/heads/main/stroke-risk-analysis/data/raw/healthcare-dataset-stroke-data.csv'

dado = pd.read_csv(url, sep=',')
dado.head(3)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1


In [4]:
dado.tail(3)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0
5109,44679,Female,44.0,0,0,Yes,Govt_job,Urban,85.28,26.2,Unknown,0


In [5]:
#verificando tipos dos dados
dado.shape
dado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [6]:
#verificando dados nulos
dado.isnull().sum()

,0
id,0
gender,0
age,0
hypertension,0
heart_disease,0
ever_married,0
work_type,0
Residence_type,0
avg_glucose_level,0
bmi,201


In [7]:
#verificando dados duplicados
dado.duplicated().sum()

np.int64(0)

In [8]:
dado.count()

,0
id,5110
gender,5110
age,5110
hypertension,5110
heart_disease,5110
ever_married,5110
work_type,5110
Residence_type,5110
avg_glucose_level,5110
bmi,4909


## 4. Limpeza e Pré-processamento
- Tratamento de valores ausentes (ex: `bmi`)
- Conversão de tipos e renomeação de colunas
- Padronização de categorias
- Criação de novas variáveis, se necessário

In [9]:
#prencher bmi com média por gênero
dado['bmi'] = dado.groupby('gender')['bmi'].transform(
    lambda x: x.fillna(x.mean())
)
dado.isnull().sum()

,0
id,0
gender,0
age,0
hypertension,0
heart_disease,0
ever_married,0
work_type,0
Residence_type,0
avg_glucose_level,0
bmi,0


In [10]:
#renomeando colunas para nomes amigáveis
dado = dado.rename(columns={
    'gender': 'genero',
    'age': 'idade',
    'hypertension': 'hipertensao',
    'heart_disease': 'doenca_cardiaca',
    'ever_married': 'casado',
    'work_type': 'tipo_trabalho',
    'Residence_type': 'tipo_residencia',
    'avg_glucose_level': 'glicose_media',
    'bmi': 'imc',
    'smoking_status': 'tabagismo',
    'stroke': 'avc'
})
dado.head(1)

,id,genero,idade,hipertensao,doenca_cardiaca,casado,tipo_trabalho,tipo_residencia,glicose_media,imc,tabagismo,avc
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1


In [11]:
#transformando colunas para categorias
col_categoria = ['genero',
    'casado',
    'tipo_trabalho',
    'tipo_residencia',
    'tabagismo']

dado[col_categoria] = dado[col_categoria].astype('category')
dado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   id               5110 non-null   int64   
 1   genero           5110 non-null   category
 2   idade            5110 non-null   float64 
 3   hipertensao      5110 non-null   int64   
 4   doenca_cardiaca  5110 non-null   int64   
 5   casado           5110 non-null   category
 6   tipo_trabalho    5110 non-null   category
 7   tipo_residencia  5110 non-null   category
 8   glicose_media    5110 non-null   float64 
 9   imc              5110 non-null   float64 
 10  tabagismo        5110 non-null   category
 11  avc              5110 non-null   int64   
dtypes: category(5), float64(3), int64(4)
memory usage: 305.3 KB


## 5. Análise Exploratória (EDA)
### 5.1 Perfil demográfico e clínico
- Distribuição de idade, gênero, IMC, glicose
- Frequência de hipertensão, doenças cardíacas

In [16]:
#segmentação por idade
dado['faixa_etaria'] = pd.cut(dado['idade'],
                              bins = [0,18,35,50,65,120],
                              labels = ['0-18', '19-35', '36-50', '51-65', '65+'])
#classificação por peso
dado['imc_categoria'] = pd.cut(dado['imc'],
                               bins = [0, 18.5, 25, 30, 35, 40, float('inf')],
                               labels=['Abaixo do peso', 'Normal', 'Sobrepeso', 'Obesidade I', 'Obesidade II', 'Obesidade III'])
#Segmentação por níveis glicemicos
dado['glicose_categoria'] = pd.cut(dado['glicose_media'],
                                   bins = [0, 99, 125, 200, float('inf')],
                                   labels=['Normal', 'Pré-diabetes', 'Diabetes', 'Extremo'])
#Simplificação de campo
dado['fumante_simples'] = dado['tabagismo'].replace({
    'formerly smoked': 'fumante',
    'smokes': 'fumante',
    'never smoked': 'não fumante',
    'Unknown': 'desconhecido'
})
#Flag de risco combinado
dado['risco_combinado'] = (
    (dado['hipertensao'] == 1) &
    (dado['doenca_cardiaca'] == 1) &
    (dado['glicose_media'] > 140)
)
#Classificar avc com nome amigavel
dado['avc_str'] = dado['avc'].replace({0: 'Sem AVC', 1: 'Com AVC'})


dado.head(3)

/tmp/ipython-input-16-732972424.py:14: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dado['fumante_simples'] = dado['tabagismo'].replace({


,id,genero,idade,hipertensao,doenca_cardiaca,casado,tipo_trabalho,tipo_residencia,glicose_media,imc,tabagismo,avc,faixa_etaria,imc_categoria,glicose_categoria,fumante_simples,risco_combinado,avc_str
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1,65+,Obesidade II,Extremo,fumante,False,Com AVC
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,29.065758,never smoked,1,51-65,Sobrepeso,Extremo,não fumante,False,Com AVC
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1,65+,Obesidade I,Pré-diabetes,não fumante,False,Com AVC


### 5.2 Comparações com a variável alvo (stroke)
- Taxa de AVC por grupo etário, sexo, hipertensão etc.
- Gráficos de comparação (barplots, boxplots, etc.)

## 6. Agregações para Visualização no Power BI
- Exportar CSVs com:
  - AVC por idade/grupo etário
  - AVC por hipertensão, glicose, IMC (categorizado)
  - Tabelas de cruzamento por gênero, tipo de trabalho, estado civil

## 7. Conclusões e próximos passos
- Sumário de insights
- Oportunidades de visualização no Power BI

# 8. Salvando arquivos para o Power BI
- `to_csv()` dos dataframes agregados